<a href="https://colab.research.google.com/github/Varad2001/Gender-classification-simple-project/blob/main/gender_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import keras 
from sklearn.model_selection import train_test_split 

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Dense, Dropout 
from keras.layers import Flatten, BatchNormalization
from tensorflow.keras.utils import to_categorical
import os
from PIL import Image

Use package 'gdown' for downloading the dataset.<br>
https://github.com/wkentaro/gdown
<br>
If file link is https://drive.google.com/file/d/1Gs6ntUJQArxq4DUUFYj3sBsjCfRof6bR/view?usp=sharing, use the file id to download that file.
Here the file id is 1Gs6ntUJQArxq4DUUFYj3sBsjCfRof6bR.
<br>
Likewise for other files.

In [6]:
# downloading the training dataset
!gdown --id file_id

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1DMkkcdZBFDr2R7YOir2z_aA9ekTSH4Pq
To: /content/train_data.csv
100% 2.22G/2.22G [00:11<00:00, 197MB/s]


In [ ]:
!ls /content

drive  sample_data  train_data.csv


In [ ]:
train_df = pd.read_csv("/content/train_data.csv")

In [ ]:
train_df.shape

(47009, 2502)

In [ ]:
# load the test data csv file 
!gdown --id file_id

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Gs6ntUJQArxq4DUUFYj3sBsjCfRof6bR
To: /content/test_data.csv
100% 550M/550M [00:04<00:00, 118MB/s]


In [ ]:
!ls /content

drive  sample_data  test_data.csv  train_data.csv


In [ ]:
test_df = pd.read_csv("/content/test_data.csv")

In [ ]:
test_df.shape

(11649, 2502)

Now we have dataframes for training and testing. <br>
Data preprocessing




In [ ]:
train_df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,result
0,0,0.050980,0.043137,0.035294,0.031373,0.015686,0.007843,0.011765,0.015686,0.047059,...,0.552941,0.450980,0.078431,0.031373,0.031373,0.015686,0.011765,0.000000,0.000000,0.0
1,1,0.039216,0.035294,0.047059,0.058824,0.101961,0.168627,0.274510,0.352941,0.423529,...,0.200000,0.372549,0.388235,0.356863,0.364706,0.321569,0.258824,0.109804,0.058824,0.0
2,2,0.776471,0.800000,0.784314,0.694118,0.611765,0.580392,0.596078,0.615686,0.611765,...,0.372549,0.396078,0.427451,0.172549,0.058824,0.172549,0.286275,0.352941,0.364706,0.0
3,3,0.176471,0.152941,0.200000,0.278431,0.356863,0.392157,0.403922,0.403922,0.443137,...,0.321569,0.250980,0.286275,0.317647,0.415686,0.290196,0.078431,0.078431,0.086275,0.0
4,4,0.211765,0.258824,0.321569,0.200000,0.192157,0.184314,0.266667,0.294118,0.250980,...,0.447059,0.482353,0.505882,0.521569,0.498039,0.466667,0.447059,0.274510,0.145098,0.0


In [ ]:
# remove the first column which is appended due to csv nature of the file
train_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,result
0,0.050980,0.043137,0.035294,0.031373,0.015686,0.007843,0.011765,0.015686,0.047059,0.125490,...,0.552941,0.450980,0.078431,0.031373,0.031373,0.015686,0.011765,0.000000,0.000000,0.0
1,0.039216,0.035294,0.047059,0.058824,0.101961,0.168627,0.274510,0.352941,0.423529,0.517647,...,0.200000,0.372549,0.388235,0.356863,0.364706,0.321569,0.258824,0.109804,0.058824,0.0
2,0.776471,0.800000,0.784314,0.694118,0.611765,0.580392,0.596078,0.615686,0.611765,0.603922,...,0.372549,0.396078,0.427451,0.172549,0.058824,0.172549,0.286275,0.352941,0.364706,0.0
3,0.176471,0.152941,0.200000,0.278431,0.356863,0.392157,0.403922,0.403922,0.443137,0.470588,...,0.321569,0.250980,0.286275,0.317647,0.415686,0.290196,0.078431,0.078431,0.086275,0.0
4,0.211765,0.258824,0.321569,0.200000,0.192157,0.184314,0.266667,0.294118,0.250980,0.247059,...,0.447059,0.482353,0.505882,0.521569,0.498039,0.466667,0.447059,0.274510,0.145098,0.0


In [ ]:
# same for test_df
test_df.drop(['Unnamed: 0'],axis=1,inplace=True)
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,result
0,0.160784,0.254902,0.356863,0.435294,0.501961,0.560784,0.619608,0.654902,0.674510,0.674510,...,0.458824,0.458824,0.454902,0.486275,0.494118,0.474510,0.478431,0.474510,0.470588,0.0
1,0.050980,0.019608,0.043137,0.082353,0.054902,0.031373,0.027451,0.027451,0.035294,0.027451,...,0.486275,0.521569,0.513725,0.490196,0.478431,0.411765,0.290196,0.196078,0.294118,0.0
2,0.070588,0.074510,0.125490,0.180392,0.262745,0.309804,0.290196,0.290196,0.294118,0.305882,...,0.286275,0.188235,0.294118,0.317647,0.384314,0.741176,0.819608,0.803922,0.811765,0.0
3,0.121569,0.121569,0.121569,0.113725,0.121569,0.141176,0.180392,0.243137,0.313725,0.368627,...,0.227451,0.160784,0.078431,0.070588,0.070588,0.101961,0.349020,0.478431,0.337255,0.0
4,0.313725,0.196078,0.164706,0.156863,0.152941,0.192157,0.180392,0.164706,0.086275,0.094118,...,0.149020,0.164706,0.196078,0.149020,0.121569,0.113725,0.105882,0.113725,0.121569,0.0


In [ ]:
# store the last column into label, the remaining dataframe is used as features
labels = train_df['result']
train_df.drop(['result'], axis=1, inplace = True)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0.050980,0.043137,0.035294,0.031373,0.015686,0.007843,0.011765,0.015686,0.047059,0.125490,...,0.392157,0.552941,0.450980,0.078431,0.031373,0.031373,0.015686,0.011765,0.000000,0.000000
1,0.039216,0.035294,0.047059,0.058824,0.101961,0.168627,0.274510,0.352941,0.423529,0.517647,...,0.062745,0.200000,0.372549,0.388235,0.356863,0.364706,0.321569,0.258824,0.109804,0.058824
2,0.776471,0.800000,0.784314,0.694118,0.611765,0.580392,0.596078,0.615686,0.611765,0.603922,...,0.380392,0.372549,0.396078,0.427451,0.172549,0.058824,0.172549,0.286275,0.352941,0.364706
3,0.176471,0.152941,0.200000,0.278431,0.356863,0.392157,0.403922,0.403922,0.443137,0.470588,...,0.396078,0.321569,0.250980,0.286275,0.317647,0.415686,0.290196,0.078431,0.078431,0.086275
4,0.211765,0.258824,0.321569,0.200000,0.192157,0.184314,0.266667,0.294118,0.250980,0.247059,...,0.392157,0.447059,0.482353,0.505882,0.521569,0.498039,0.466667,0.447059,0.274510,0.145098


In [ ]:
# same for testing data
y_test = test_df['result']
test_df.drop(['result'],axis=1,inplace = True)
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0.160784,0.254902,0.356863,0.435294,0.501961,0.560784,0.619608,0.654902,0.674510,0.674510,...,0.423529,0.458824,0.458824,0.454902,0.486275,0.494118,0.474510,0.478431,0.474510,0.470588
1,0.050980,0.019608,0.043137,0.082353,0.054902,0.031373,0.027451,0.027451,0.035294,0.027451,...,0.482353,0.486275,0.521569,0.513725,0.490196,0.478431,0.411765,0.290196,0.196078,0.294118
2,0.070588,0.074510,0.125490,0.180392,0.262745,0.309804,0.290196,0.290196,0.294118,0.305882,...,0.415686,0.286275,0.188235,0.294118,0.317647,0.384314,0.741176,0.819608,0.803922,0.811765
3,0.121569,0.121569,0.121569,0.113725,0.121569,0.141176,0.180392,0.243137,0.313725,0.368627,...,0.239216,0.227451,0.160784,0.078431,0.070588,0.070588,0.101961,0.349020,0.478431,0.337255
4,0.313725,0.196078,0.164706,0.156863,0.152941,0.192157,0.180392,0.164706,0.086275,0.094118,...,0.137255,0.149020,0.164706,0.196078,0.149020,0.121569,0.113725,0.105882,0.113725,0.121569


In [ ]:
# train_df contains the features for training the model and 'labels' contains the corresponding labels/results

Now convert each image array into 3-d array of shape (50,50,1) to represent the pixels. <br>
Thus the train_df should be converted into shape (no_of_images,50,50,1).

In [ ]:
# reshaping can only be performed on numpy arrays . Hence , convert the train_df and test_df into numpy arrays.
train_arr = np.array(train_df)
test_arr = np.array(test_df)

In [ ]:
rows, cols = 50, 50
train_arr = train_arr.reshape(train_df.shape[0],rows, cols, 1)
test_arr = test_arr.reshape(test_df.shape[0],rows, cols, 1)

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_arr, train_y, test_size=0.2)

In [ ]:
batch_size = 256
epochs = 50
input_shape = (rows,cols,1)

<h6> The CNN model</h6>
This is a typical CNN model; taken from following source : https://www.freecodecamp.org/news/creating-your-first-image-classifier/
<br>
As the model produces output vector of size 2, the 'unit' parameter for Dense method is set to 2. Else is left unchanged.

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model

In [ ]:
# training the model
model = baseline_model()
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
history = model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(val_x, val_y))

Epoch 1/50
147/147 [==============================] - 130s 874ms/step - loss: 0.6275 - accuracy: 0.6853 - val_loss: 0.6506 - val_accuracy: 0.6643
Epoch 2/50
147/147 [==============================] - 129s 878ms/step - loss: 0.4500 - accuracy: 0.7940 - val_loss: 0.4779 - val_accuracy: 0.8532
Epoch 3/50
147/147 [==============================] - 132s 900ms/step - loss: 0.3877 - accuracy: 0.8294 - val_loss: 0.3351 - val_accuracy: 0.8884
Epoch 4/50
147/147 [==============================] - 130s 882ms/step - loss: 0.3554 - accuracy: 0.8463 - val_loss: 0.2836 - val_accuracy: 0.8963
Epoch 5/50
147/147 [==============================] - 129s 877ms/step - loss: 0.3345 - accuracy: 0.8584 - val_loss: 0.2710 - val_accuracy: 0.8982
Epoch 6/50
147/147 [==============================] - 127s 865ms/step - loss: 0.3225 - accuracy: 0.8625 - val_loss: 0.2545 - val_accuracy: 0.9060
Epoch 7/50
147/147 [==============================] - 133s 904ms/step - loss: 0.3110 - accuracy: 0.8690 - val_loss: 0.2360 -

In [ ]:
# create directory to save the model
!ls /content/saved_model

my_model2


In [ ]:
# save the model
model.save("my_model2")

INFO:tensorflow:Assets written to: saved_model/my_model2/assets


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 50, 50, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 50, 50, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 25, 25, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 25, 25, 32)       128       
 hNormalization)                                                 
                                                        